## 1) Parametros

In [1]:
#links hechos durante el pipeline
#apps script
#https://script.google.com/d/1UCOMAWc8WzM6L8pdtKByfcOWjqr5YMa6lRDc3pDrA-9LFf9GmVqXuCpR/edit?usp=sharing
#formulario 1 URL:  https://docs.google.com/forms/d/e/1FAIpQLSfbh_Hh0NfIAcszWXEiK7Ge_mas523DR8HmjXVlKL_u6BGFWQ/viewform?usp=sharing
#formulario 2 URL:  https://docs.google.com/forms/d/e/1FAIpQLScmHnd1tYezOL3QU5DYyp7NGleguppm-alBUqw84N9Ttki1rQ/viewform?usp=sharing
#formulario 3 URL:  https://docs.google.com/forms/d/e/1FAIpQLSfv0s0cThUGi2GoXLKYnRvJnvcqmcziK1h5KapqiITR5o040A/viewform?usp=sharing

#Entrada
apps_script_url = "https://script.google.com/macros/s/AKfycbyo9CLuK5nJYk3dIZDworpXKYPvRaA_sXDA8mcW4Ai3OBpKM9_qrXe0myUkhOOdZW-N/exec" #appscript
carpeta_con_datos = "Ejemplo Pipeline/Entradas/"
nombre_de_archivo = "Textos.csv" #csv
plantillas_url = "https://docs.google.com/spreadsheets/d/1N8GQpIxZLjPYCJGrqpoymxqQsDoPjb-vQ621Tbcf13c/edit?usp=sharing" #google sheets

carpeta_raiz_drive = "/content/drive/My Drive/"

#Salida
carpeta_guardar_salidas = "Ejemplo Pipeline/Salidas/"
nombre_formularios_archivo = "Ejemplo Formulario Pipeline"
nombre_formularios = "Clasificacion de textos"
titulo_formularios = "Clasificacion de textos en ingles"
descripcion_formulario = "Este formulario forma parte de un ejemplo para el pipeline de evaluacion de datos generados por I.A. (https://github.com/juan-oviedo/PipelineEvaluacion)"
nombre_csv_con_formularios = "Links_de_formularios.csv" #csv

cantidad_de_paquetes = 3
cantidad_de_superposiciones = 2
cantidad_de_datos_a_evaluar = 10
seed = 23

## 2) Imports

In [2]:
import random
from google.colab import drive
import os
import pandas as pd
import requests
import sklearn.model_selection as sk
import time
import gspread
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build

## 3) Personalizacion y funciones

### a) Funcion para generar paquetes de indices


In [3]:
#Se generan N paquetes.
#Cada dato va a aparecer en M paquetes.
def create_balanced_index_packs(indices, N, M):
    random.shuffle(indices)
    # Initialize empty packs and track their sizes
    packs = [[] for _ in range(N)]
    pack_sizes = [0] * N  # Track the number of texts in each pack

    for indice in indices:
        indexes = []
        # Find the average size of the packs
        avg_pack_size = sum(pack_sizes) / N

        attempts = 0
        max_attempts = 10 * (N + M)  # Set a reasonable limit for attempts

        # Select M unique packs that have sizes below or close to the average
        while len(indexes) < M:
            index = random.randint(0, N - 1)
            attempts += 1
            # Only add this index if it's not already chosen and if the pack is balanced
            if index not in indexes and pack_sizes[index] <= avg_pack_size:
                indexes.append(index)

            # Relax condition if too many attempts
            elif attempts > max_attempts and index not in indexes:
                indexes.append(index)
                attempts = 0  # Reset attempts for the next text

        # Distribute the text to the selected packs
        for index in indexes:
            packs[index].append(indice)
            pack_sizes[index] += 1  # Update the pack size

    return packs

### b) Funciones para la generacion de paquetes de datos

In [4]:
#genera una lista de paquetes
#cada paquete tiene una lista de datos a evaluar
def generar_lista_de_paquetes (matriz_de_indices):
  lista_de_paquetes_de_datos = []
  for lista_de_indices in matriz_de_indices:
    paquetes_de_textos = generar_paquete(lista_de_indices)
    lista_de_paquetes_de_datos.append(paquetes_de_textos)
  return lista_de_paquetes_de_datos

def generar_paquete(lista_de_indices):
  paquetes_de_datos = []
  for indice in lista_de_indices:
    dato = datos[indice]
    paquetes_de_datos.append(dato)
  return paquetes_de_datos

### c) Funciones para las plantillas

In [5]:
#funciones para agregar los diferentes items
def add_secction(title, desc=''):
  tipo.append(0)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append('')

def add_title(title, desc=''):
  tipo.append(1)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append('')

def add_choice(title, val, oblig, desc=''):
  tipo.append(2)
  titulo.append(title)
  descripcion.append(desc)
  valores.append(val)
  obligatorio.append(oblig)

def add_paragraph(title, desc='', oblig=False):
  tipo.append(3)
  titulo.append(title)
  descripcion.append(desc)
  valores.append('')
  obligatorio.append(oblig)

In [6]:
def ejecutar_plantilla(informacion, plantilla):
  lista_de_titulos = informacion[0]
  lista_de_descripciones = informacion[1]
  i = 0; #titulos
  j = 0; #descripciones
  for row in plantilla.itertuples():

    #agregar una seccion
    if row.Tipo_de_elemento == 'Seccion':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_secction(lista_de_titulos[i], lista_de_descripciones[j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_secction(lista_de_titulos[i], row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_secction(row.titulo, lista_de_descripciones[j])
        j += 1
      else:
        add_secction(row.titulo, row.descripcion)

    #agregar un titulo
    elif row.Tipo_de_elemento == 'Titulo':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_title(lista_de_titulos[i], lista_de_descripciones[j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_title(lista_de_titulos[i], row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_title(row.titulo, lista_de_descripciones[j])
        j += 1
      else:
        add_title(row.titulo, row.descripcion)

    #Agregar un multiple Choice
    elif row.Tipo_de_elemento == 'Multiple Choice':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_choice(lista_de_titulos[i], row.valores, row.es_obligatorio, lista_de_descripciones[j])
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_choice(lista_de_titulos[i], row.valores, row.es_obligatorio, row.descripcion)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_choice(row.titulo, row.valores, row.es_obligatorio, lista_de_descripciones[j])
        j += 1
      else:
        add_choice(row.titulo, row.valores, row.es_obligatorio, row.descripcion)

    #Agregar un parrafo
    elif row.Tipo_de_elemento == 'Parrafo':
      if row.titulo_se_rellena_en_ejecucion == "Si" and row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_paragraph(lista_de_titulos[i], lista_de_descripciones[j], row.es_obligatorio)
        i += 1
        j += 1
      elif row.titulo_se_rellena_en_ejecucion == "Si":
        add_paragraph(lista_de_titulos[i], row.descripcion, row.es_obligatorio)
        i += 1
      elif row.descripcion_se_rellena_en_tiempo_de_ejecucion == "Si":
        add_paragraph(row.titulo, lista_de_descripciones[j], row.es_obligatorio)
        j += 1
      else:
        add_paragraph(row.titulo, row.descripcion, row.es_obligatorio)


    else:
      print("Elemento no reconocido:", row.Tipo)


### d) Orden de las Plantillas
aca es donde mas se puede personalizar los formularios, ya que se pueden generar varias plantillas y darle el orden que uno desee.

In [7]:
#ej:
def orden_plantillas(pack):
  cantidad_elementos = 0
  #se ejecuta por cada dato en un paquete
  for j, dato in enumerate(pack):
    #informacion con la que se tiene que rellenar los titulos de algunos elementos
    #ej:
    titulos = [f"Texto {j+1} de {len(pack)}"]

    #informacion con la que se tiene que rellenar las descripciones de algunos elementos
    descripciones = [dato]

    informacion = [titulos, descripciones]

    #se comunica atravez de las listas tipo, titulo, descripcion, valores, obligatorio
    ejecutar_plantilla (informacion, plantillas[0])

    #sumamos los elementos de la primer plantilla
    cantidad_elementos += plantillas[0].shape[0]
  return cantidad_elementos

### e) Funcion para realizar request a Apps script

In [8]:
def make_request(url, params):
    """Send a GET request to the Apps Script and process the response."""
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            try:
                result = response.json()  # Parse JSON response
                return result.get('url'), result.get('formId')
            except requests.exceptions.JSONDecodeError:
                print("Response is not valid JSON:", response.text)
                return None, None
        else:
            print(f"Failed to connect: {response.status_code}")
            return None, None
    except requests.RequestException as e:
        print(f"Error during request: {e}")
        return None, None

## 4) Pipeline

In [15]:
#Montar google drive

drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Cargar datos

# definir ruta de los datos de entrada
ruta_datos = carpeta_raiz_drive + carpeta_con_datos

# chekear si la ruta existe
if not os.path.exists(ruta_datos):
    print(f"Folder at {ruta_datos} dont exist")

csv_file_path = ruta_datos + nombre_de_archivo
df = pd.read_csv(csv_file_path)

#datos a validar ej:
datos = df['text'].tolist()

In [17]:
# Crear o abrir carpeta de salida

# definir ruta de los datos de salida
ruta_salida = carpeta_raiz_drive + carpeta_guardar_salidas

# Crear carpeta si no existe
if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)
    print(f"Folder created at {ruta_salida}")
else:
    print(f"Folder already exists at {ruta_salida}")

Folder already exists at /content/drive/My Drive/Ejemplo Pipeline/Salidas/


In [18]:
#Crear paquetes de indices

#Seteamos la semilla
random.seed(seed)
n = len(datos)

#obtenemos los indices de los datos a evaluar
random_indices = random.sample(range(n), cantidad_de_datos_a_evaluar)

index_packs = create_balanced_index_packs(random_indices, cantidad_de_paquetes, cantidad_de_superposiciones)
for idx, pack in enumerate(index_packs):
    print(f"Pack {idx + 1}: {pack}")

Pack 1: [114, 118, 54, 75, 99, 39, 2]
Pack 2: [118, 106, 37, 54, 99, 10]
Pack 3: [114, 106, 37, 75, 10, 39, 2]


In [19]:
#Crear paquetes de textos

text_packs = generar_lista_de_paquetes(index_packs)

In [20]:
#Buscar plantillas

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open the spreadsheet
spreadsheet = gc.open_by_url(plantillas_url)

# Loop through all worksheets
worksheets = spreadsheet.worksheets()
dfs = {}

for ws in worksheets:
    data = ws.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    dfs[ws.title] = df

plantillas = []
for sheet_name, plantilla in dfs.items():
  plantillas.append(plantilla)

In [21]:
#Crear CSVs con la informacion de los formularios

data_form = []
paquetes = []
#se crea un archivo csv por paquete
for i, pack in enumerate(text_packs):
  file_name = nombre_formularios_archivo + str(i+1) + '.csv'

  tipo = []
  titulo = []
  descripcion = []
  valores = []
  obligatorio = []

  #Se cuenta la cantidad de elementos en total por paquete
  cantidad_elementos = orden_plantillas(pack)

  data = {
      'Tipo': tipo,
      'Titulo': titulo,
      'Descripcion': descripcion,
      'Valores': valores,
      'Obligatorio': obligatorio
  }


  #se crea un data frame con toda la informacion del formulario
  df = pd.DataFrame(data)

  #se guarda como un csv
  csv_file_path = ruta_salida + file_name
  df.to_csv(csv_file_path, sep='¬', lineterminator='~', index=False)

  #Informacion sobre el formulario
  Nombre_form = nombre_formularios + str(i+1)
  titulo_form = titulo_formularios + str(i+1)
  url = ''
  datos_del_formulario = [file_name, Nombre_form, titulo_form, descripcion_formulario, url , cantidad_elementos]
  data_form.append(datos_del_formulario)
  print(f"CSV file created at {csv_file_path}")

  paquetes.append(titulo_form)

CSV file created at /content/drive/My Drive/Ejemplo Pipeline/Salidas/Ejemplo Formulario Pipeline1.csv
CSV file created at /content/drive/My Drive/Ejemplo Pipeline/Salidas/Ejemplo Formulario Pipeline2.csv
CSV file created at /content/drive/My Drive/Ejemplo Pipeline/Salidas/Ejemplo Formulario Pipeline3.csv


In [22]:
#Esperar que se carguen los CSV en drive

time.sleep(15)

In [23]:
#Creacion de formularios a traves de Apps Script

form_urls = []
form_ids = []

for data in data_form:
  data[4] = ''
  cantidad_elementos = data[5]
  for j in range ((cantidad_elementos//10) + 1):
    params = {
        'folderPath': carpeta_guardar_salidas,
        'fileName': data[0],
        'formName': data[1],
        'title': data[2],
        'descipcion': data[3],
        'formId': data[4],
        'startElement': 10*j,
        'stopElement': 10*(j+1) - 1
    }

    form_url, form_id = make_request(apps_script_url, params)
    data[4] = form_id

    if not form_url:
      print("Failed to retrieve form details.")
      break
    if j >= (cantidad_elementos//10):
      print("URL: ", form_url)
      print("Id: ", form_id)

  form_urls.append(form_url)
  form_ids.append(form_id)

URL:  https://docs.google.com/forms/d/1Wr5LftJDCDgZBxVdpaKIohoAZc5ot4mMQzwNBF9CfX0/edit
Id:  1Wr5LftJDCDgZBxVdpaKIohoAZc5ot4mMQzwNBF9CfX0
URL:  https://docs.google.com/forms/d/1EWy3sPLVW-KgrlirI-rYZbBoIj6BdhflAcROarc3_Nw/edit
Id:  1EWy3sPLVW-KgrlirI-rYZbBoIj6BdhflAcROarc3_Nw
URL:  https://docs.google.com/forms/d/1sjYmTO160Ljw31_hcFFp9h-AtI6vCWxDCPbNFTGhXy8/edit
Id:  1sjYmTO160Ljw31_hcFFp9h-AtI6vCWxDCPbNFTGhXy8


In [24]:
#Crear csv con links de los formularios

data_formularios = {
      'Paquete': paquetes,
      'Links': form_urls,
      'Form ID': form_ids
  }

df = pd.DataFrame(data_formularios)
csv_file_path = ruta_salida + nombre_csv_con_formularios
df.to_csv(csv_file_path, index=False)
print(f"CSV file created at {csv_file_path}")

CSV file created at /content/drive/My Drive/Ejemplo Pipeline/Salidas/Links_de_formularios.csv
